In [ ]:
import polars as pl 
import polars.selectors as cs
import seaborn as sns
import utils
import get_data
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.manifold import TSNE
import itertools

In [ ]:
frame = pl.read_parquet(r"C:\Users\faune\Downloads\0000.parquet")

In [ ]:
to_compare = (
    frame
    .drop('idx')
    .with_columns(pl.col('label')
    .replace({0:'negative', 1: 'positive'}))
    .sort('label', descending=True)
    .unstack(step=37569, how="vertical", columns='sentence')
    .rename({'sentence_0': 'positive', 'sentence_1':'negative'})
)

In [ ]:
negative = to_compare.select('negative').drop_nulls().to_series().to_list()
positive = to_compare.select('positive').drop_nulls().to_series().to_list()

In [ ]:
map_neg = map(lambda x: set(x.strip().split(' ')), negative)
map_pos = map(lambda x: set(x.strip().split(' ')), positive)

In [101]:
to_compare

positive,negative
str,str
"""that loves its…","""hide new secre…"
"""demonstrates t…","""contains no wi…"
"""of saucy ""","""remains utterl…"
"""are more deepl…","""on the worst r…"
"""the greatest m…","""that 's far to…"
…,…
"""at least passa…",null
"""showing off hi…",null
"""a delightful c…",null


In [ ]:
# list(filter(lambda i: len(i.intersection(list(map_neg)[0]))==max(map(lambda x,y : len(x.intersection(y)), itertools.repeat(list(map_neg)[0]), map_pos)), map_pos))

In [ ]:
for i in map_neg:   
    print(list(filter(
        lambda y: len(y.intersection(i))==max(map(lambda x : len(x.intersection(i)),  map_pos)), 
        map_pos)))

In [ ]:
for i in map_neg:
    to_match = set(i.strip().split(' '))
    for j in positive:
        set(i.strip().split(' '))

In [98]:
map_neg = map(lambda x: {'sentence' :x, 'set': set(x.strip().split(' '))}, negative)
map_pos = map(lambda x: {x : set(x.strip().split(' '))}, positive)
# max_intersection_length = max(len(i.intersection(j)) for i in map_neg for j in map_pos)
# result = [j for i in map_neg for j in map_pos if len(i.intersection(j)) == max_intersection_length]


In [100]:
pl.DataFrame(list(map_neg))

shape: (0, 0)
┌┐
╞╡
└┘

In [ ]:
similar_sentences = similar_sentences.with_columns(
            pl.Series(
                name= 'positive_embeddings', 
                values= get_data._get_embeddings_by_chunks(similar_sentences.select('positive').to_series().to_list(), 50)
            )
        ).with_columns(
            pl.Series(
                name= 'negative_embeddings', 
                values= get_data._get_embeddings_by_chunks(similar_sentences.select('negative').to_series().to_list(), 50)
            )
        )

In [ ]:
similar_sentences = (
    similar_sentences
    .with_columns(
        similar_sentences.with_columns(pl.col('positive_embeddings').list.to_array(1024),pl.col('negative_embeddings').list.to_array(1024))
        .map_rows(function=lambda t : tuple(cosine_similarity(X=np.array(t[10]).reshape(1, -1), Y=np.array(t[11]).reshape(1, -1)).tolist()))
    )
    .explode('column_0')
    .sort('column_0')
    .rename({'column_0': 'cosine_similarity'})
    .with_columns(
        similar_sentences.with_columns(pl.col('positive_embeddings').list.to_array(1024),pl.col('negative_embeddings').list.to_array(1024))
        .map_rows(function=lambda t :np.dot(a=t[10], b=np.transpose(t[11])))
    )
    .rename({'map': 'dot_product'})
    .sort('cosine_similarity')
    # .sort(['dot_product', 'cosine_similarity'])
)